### Import all the modules and dependencies

In [ ]:
import cv2
import time 
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output

### All the functions for the different components

#### Fish Eye Removal

In [ ]:
def process_per_frame_fisheye(frame, width, height):
    """
    Correct fisheye distortion in a frame using given calibration parameters.
    Args:
        frame: Input distorted frame.
        width, height: Dimensions of the frame.
    Returns:
        corrected_frame: Undistorted frame.
    """
    # Normalized intrinsic parameters
    cx, cy = 0.5, 0.5  # Center coordinates (normalized)
    # k1, k2 = -0.227, -0.022  # Radial distortion coefficients
    k1, k2 = -0.230, -0.020  # Experiment with small adjustments


    # Camera intrinsic matrix
    cx_px = cx * width
    cy_px = cy * height
    K = np.array([
        [width, 0, cx_px],
        [0, height, cy_px],
        [0, 0, 1]
    ], dtype=np.float32)

    # Distortion coefficients
    D = np.array([k1, k2, 0.09, 0], dtype=np.float32)

    # Undistortion map for fisheye
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(
        K, D, np.eye(3), K, (width, height), cv2.CV_16SC2
    )

    # Apply remapping
    corrected_frame = cv2.remap(frame, map1, map2, interpolation=cv2.INTER_LINEAR)
    return corrected_frame

#### Split into sections

In [ ]:
def split_image_into_thirds(img):
        # Get image dimensions
        height, width = img.shape[:2]
        
        # Calculate the width of each section
        section_width = width // 3
        
        # Split the image into three sections
        left_section = img[:, :section_width]
        middle_section = img[:, section_width:section_width*2]
        right_section = img[:, section_width*2:]
        
        return left_section, middle_section, right_section

#### Head Detection

In [ ]:
     

def detect_head(img):
    try:
        plt.imshow(img)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detect faces (both frontal and profile)
        frontal_faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        profile_faces = profile_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        # Combine detections
        all_detections = list(frontal_faces) + list(profile_faces)
        
        return len(all_detections) > 0, len(all_detections)
        
    except Exception as e:
        print(f"Error processing image: {str(e)}")
        return False, 0, None

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')   

def detect_head(img):
    try:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detect faces (both frontal and profile)
        frontal_faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        profile_faces = profile_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        # Combine detections
        all_detections = list(frontal_faces) + list(profile_faces)
        
        # Create copy for visualization
        img_with_detections = img.copy()
        
        # Draw rectangles around detected heads
        for (x, y, w, h) in all_detections:
            cv2.rectangle(img_with_detections, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(img_with_detections, 'Head', (x, y-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        percent_img = 0.8
        _img_with_detections = cv2.resize(img_with_detections, 
                                          (int(img_with_detections.shape[1]*percent_img), 
                                           int(img_with_detections.shape[0]*percent_img))
                                         )
        
        # # Create window with specific size
        # cv2.namedWindow('Head Detection Result', cv2.WINDOW_NORMAL)
        # cv2.resizeWindow('Head Detection Result', 640, 480)
        # cv2.imshow('Head Detection Result', _img_with_detections)
 
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        
        heads_found = len(all_detections) > 0
        num_heads = len(all_detections)
        
        return heads_found, num_heads, img_with_detections
        
    except Exception as e:
        print(f"Error processing image: {str(e)}")
        return False, 0, None

## Video Processing

### File Based Processing

In [ ]:
# Path to your video file
video_path = "./GX010544"
# video_path = "./GX010542.MP4"

In [ ]:
# Open the video file
cap = cv2.VideoCapture(video_path)
# # Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    """ For Resizing - viewing purpose """
    _frame = cv2.resize(frame, (640, 480))
    # Create window with specific size
    cv2.namedWindow('Video', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Video', 640, 480)
    cv2.imshow('Video', _frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# cap.release()
cv2.destroyAllWindows()

In [ ]:
ret, frame = cap.read()
if ret:
    # Convert the frame from BGR (OpenCV format) to RGB (matplotlib format)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(frame_rgb)

In [ ]:
linear_frame = process_per_frame_fisheye(frame_rgb, width, height)
plt.imshow(linear_frame)

In [ ]:
l, m, r = split_image_into_thirds(linear_frame)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
# Display each section
ax1.imshow(l)
ax1.set_title('Left Section')
ax1.axis('off')

ax2.imshow(m)
ax2.set_title('Middle Section')
ax2.axis('off')

ax3.imshow(r)
ax3.set_title('Right Section')
ax3.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
l_bool, l_count, _l = detect_head(l)
m_bool, m_count, _m = detect_head(m)
r_bool, r_count, _r = detect_head(r)

print(f"Left: ({l_bool}, {l_count}); Middle: ({m_bool}, {m_count}); Right:({r_bool}, {r_count})")

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
# Display each section
ax1.imshow(_l)
ax1.set_title('Left Section')
ax1.axis('off')

ax2.imshow(_m)
ax2.set_title('Middle Section')
ax2.axis('off')

ax3.imshow(_r)
ax3.set_title('Right Section')
ax3.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
cap.release()

### Accessing from GoPro Feed

In [ ]:
import cv2
from matplotlib import pyplot as plt
from goprocam import GoProCamera, constants

In [ ]:
cam_cap = cv2.VideoCapture("udp://127.0.0.1:8081")

In [ ]:
cam_cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# Additional settings/ to minimize latency
cam_cap.set(cv2.CAP_PROP_POS_FRAMES, 0) 

In [ ]:
while True:
    # Clear buffer
    while cam_cap.grab():
        pass
    
    # Read latest frame
    ret, frame = cam_cap.read()
    if ret:
        cv2.imshow('Latest UDP Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

In [ ]:
_, check_frame = cam_cap.read()

In [ ]:
_

In [ ]:
plt.imshow(cv2.cvtColor(check_frame, cv2.COLOR_BGR2RGB))

## Accessing GoPro using SDK

In [ ]:
from goprocam import GoProCamera, constants

In [ ]:
gopro = GoProCamera.GoPro(constants.gpcontrol)

In [ ]:
gopro.overview()

In [ ]:
gopro.take_photo()

In [ ]:
gopro.downloadLastMedia(custom_filename='check12')